# Motion Vector Generator - 4 Parameters

### General parameters

In [67]:
###################################################
# Variables setup for the MV generator
###################################################
import math
from datetime import date

COORD_WIDTH = 8
MV_WIDTH = 8
W = 4                      # Block width
H = 128                      # Block height

### 4-Parameter MV Generator

In [32]:
#########################################
# Variables calulation
#########################################
FRAC_WIDTH = int(math.log2(W))
print('Width of the fractional portion:', FRAC_WIDTH, 'bits')

REPRESENTATION = ['.']
for i in range(FRAC_WIDTH):
    REPRESENTATION.append('x')
REPRESENTATION_CONCAT = ''.join(REPRESENTATION)

MV_0_H_EXT = []
MV_0_V_EXT = []
for j in range(COORD_WIDTH-2):
    MV_0_H_EXT.append('MV_0_H['+str(MV_WIDTH-1)+']')
    MV_0_V_EXT.append('MV_0_V['+str(MV_WIDTH-1)+']')
MV_0_H_EXT_CONCAT = ', '.join(MV_0_H_EXT)
MV_0_V_EXT_CONCAT = ', '.join(MV_0_V_EXT)

# print(MV_0_H_EXT_CONCAT)
# print(MV_0_V_EXT_CONCAT)
# print(REPRESENTATION_CONCAT)

Width of the fractional portion: 4 bits


In [34]:
###############################################
# Text writing: Mux generic
###############################################

mv_gen_4_text = '''/*--------------------------------------------------------------------
 * File: MV_gen_4_param.v
 * Date generated: 2023-04-12
 * Date modified: '''+str(date.today())+'''
 * Author: Bruna Suemi Nagai
 * Description: Motion Vectors generator for 4-parameters
 *------------------------------------------------------------------------------ */

module MV_gen(
	X_COORD,				// The X coordinate of the sample to be reconstructed [Fast Affine ME for VVC, Park S.]
	Y_COORD,				// The Y coordinate of the sample to be reconstructed [Fast Affine ME for VVC, Park S.]	
	MV_0_H,					// Horizontal component for MV0
	MV_1_H,					// Vertical component for MV0	
	MV_0_V,					// Horizontal component for MV0
	MV_1_V,					// Vertical component for MV1	
	MV_H_OUT,				// Horizontal component for the resultant MV
	MV_V_OUT				// Vertical component for the resultant MV	
);


// ------------------------------------------
// IO declaration
// ------------------------------------------
	input signed ['''+str(COORD_WIDTH-1)+''':0] X_COORD;
	input signed ['''+str(COORD_WIDTH-1)+''':0] Y_COORD;
	
	input signed ['''+str(MV_WIDTH-1)+''':0] MV_0_H;
	input signed ['''+str(MV_WIDTH-1)+''':0] MV_1_H;
	
	input signed ['''+str(MV_WIDTH-1)+''':0] MV_0_V;
	input signed ['''+str(MV_WIDTH-1)+''':0] MV_1_V;

	output wire signed ['''+str(MV_WIDTH+COORD_WIDTH+2)+''':0] MV_H_OUT;
	output wire signed ['''+str(MV_WIDTH+COORD_WIDTH+2)+''':0] MV_V_OUT;
	

// ------------------------------------------
// Signals definitions
// ------------------------------------------
	wire signed ['''+str(MV_WIDTH-1)+''':0] MINUS_MV_0_H;
	wire signed ['''+str(MV_WIDTH-1)+''':0] MINUS_MV_0_V;

	wire signed ['''+str(MV_WIDTH)+''':0] SUB_H;
	wire signed ['''+str(MV_WIDTH)+''':0] SUB_V;
	
	wire signed ['''+str(MV_WIDTH+COORD_WIDTH)+''':0] MULT_H_H;		// 9 bits * 8 bits = 17 bits
	wire signed ['''+str(MV_WIDTH+COORD_WIDTH)+''':0] MULT_H_V;		// 9 bits * 8 bits = 17 bits
	wire signed ['''+str(MV_WIDTH+COORD_WIDTH)+''':0] MULT_V_H;		// 9 bits * 8 bits = 17 bits
	wire signed ['''+str(MV_WIDTH+COORD_WIDTH)+''':0] MULT_V_V;		// 9 bits * 8 bits = 17 bits
	
	wire signed ['''+str(MV_WIDTH+COORD_WIDTH+1)+''':0] SUM_H;
	wire signed ['''+str(MV_WIDTH+COORD_WIDTH+1)+''':0] SUM_V;
	
	wire signed ['''+str(MV_WIDTH+COORD_WIDTH+1)+''':0] MV_0_H_EXT;
	wire signed ['''+str(MV_WIDTH+COORD_WIDTH+1)+''':0] MV_0_V_EXT;
	
	
// ------------------------------------------
// Logic
// ------------------------------------------
	assign MINUS_MV_0_H = ~MV_0_H + '''+str(MV_WIDTH)+''''b1;
	assign MINUS_MV_0_V = ~MV_0_V + '''+str(MV_WIDTH)+''''b1; 

	assign SUB_H = MV_1_H + MINUS_MV_0_H;	
	assign SUB_V = MV_1_V + MINUS_MV_0_V;
	
	assign MULT_H_H = SUB_H * X_COORD;
	assign MULT_H_V = SUB_V * Y_COORD; 
	
	assign MULT_V_H = SUB_H * Y_COORD;
	assign MULT_V_V = SUB_V * X_COORD; 

	// Considerar que na divisão por '''+str(W)+''', o ponto está fixo em '''+REPRESENTATION_CONCAT+''' (>> '''+str(FRAC_WIDTH)+''')

	assign SUM_H = MULT_H_H + MULT_H_V; 
	assign MV_0_H_EXT = {'''+MV_0_H_EXT_CONCAT+''', MV_0_H, '''+str(FRAC_WIDTH)+''''b0};
	
	assign SUM_V = MULT_V_H + MULT_V_V; 
	assign MV_0_V_EXT = {'''+MV_0_V_EXT_CONCAT+''', MV_0_V, '''+str(FRAC_WIDTH)+''''b0};
	
	assign MV_H_OUT = SUM_H + MV_0_H_EXT;
	assign MV_V_OUT = SUM_V + MV_0_V_EXT;
	
endmodule
'''

# print(mv_gen_4_text)

### 6-Parameters MV Generator

In [82]:
#######################################################
# Calculation for fixed point representations
#######################################################
FRAC_W_WIDTH = int(math.log2(W))
print('Width of the fractional portion for W:', FRAC_W_WIDTH, 'bits')

FRAC_H_WIDTH = int(math.log2(H))
print('Width of the fractional portion for H:', FRAC_H_WIDTH, 'bits')

REPRESENTATION_W = ['.']
for i in range(FRAC_W_WIDTH):
    REPRESENTATION_W.append('x')
REPRESENTATION_W_CONCAT = ''.join(REPRESENTATION_W)

REPRESENTATION_H = ['.']
for k in range(FRAC_H_WIDTH):
    REPRESENTATION_H.append('x')
REPRESENTATION_H_CONCAT = ''.join(REPRESENTATION_H)


#######################################################
# Generating an extended wire for additions
#######################################################
if H > W:
    print('H > W')
    GREATER_FRAC = FRAC_H_WIDTH
    MULT_H_0_EXT_CONCAT = "{MULT_H_0, " + str(int(FRAC_H_WIDTH-FRAC_W_WIDTH)) + "'b0} + "
    MULT_H_1_EXT = ['{']
    for b in range(int(FRAC_H_WIDTH-FRAC_W_WIDTH)):
        MULT_H_1_EXT.append('MULT_H_1[' + str(MV_WIDTH+COORD_WIDTH) + '], ')
    MULT_H_1_EXT.append('MULT_H_1};')
    MULT_H_1_EXT_CONCAT = ''.join(MULT_H_1_EXT)
    SUM_H_WIDTH = MV_WIDTH + COORD_WIDTH + 1 + (FRAC_H_WIDTH - FRAC_W_WIDTH)
    ###########################################################################
    MULT_V_0_EXT_CONCAT = "{MULT_V_0, " + str(int(FRAC_H_WIDTH-FRAC_W_WIDTH)) + "'b0} + "
    MULT_V_1_EXT = ['{']
    for d in range(int(FRAC_H_WIDTH-FRAC_W_WIDTH)):
        MULT_V_1_EXT.append('MULT_V_1[' + str(MV_WIDTH+COORD_WIDTH) + '], ')
    MULT_V_1_EXT.append('MULT_V_1};')
    MULT_V_1_EXT_CONCAT = ''.join(MULT_V_1_EXT)
    SUM_V_WIDTH = MV_WIDTH + COORD_WIDTH + 1 + (FRAC_H_WIDTH - FRAC_W_WIDTH)
elif W > H:
    print('W > H')
    GREATER_FRAC = FRAC_W_WIDTH
    MULT_H_1_EXT_CONCAT = " + {MULT_H_1, " + str(int(FRAC_W_WIDTH-FRAC_H_WIDTH)) + "'b0};"
    MULT_H_0_EXT = ['{']
    for c in range(int(FRAC_W_WIDTH-FRAC_H_WIDTH)):
        MULT_H_0_EXT.append('MULT_H_0[' + str(MV_WIDTH+COORD_WIDTH) + '], ')
    MULT_H_0_EXT.append('MULT_H_0}')
    MULT_H_0_EXT_CONCAT = ''.join(MULT_H_0_EXT)
    SUM_H_WIDTH = MV_WIDTH + COORD_WIDTH + 1 + (FRAC_W_WIDTH - FRAC_H_WIDTH)
    ###########################################################################
    MULT_V_1_EXT_CONCAT = " + {MULT_V_1, " + str(int(FRAC_W_WIDTH-FRAC_H_WIDTH)) + "'b0};"
    MULT_V_0_EXT = ['{']
    for e in range(int(FRAC_W_WIDTH-FRAC_H_WIDTH)):
        MULT_V_0_EXT.append('MULT_V_0[' + str(MV_WIDTH+COORD_WIDTH) + '], ')
    MULT_V_0_EXT.append('MULT_V_0}')
    MULT_V_0_EXT_CONCAT = ''.join(MULT_V_0_EXT)
    SUM_V_WIDTH = MV_WIDTH + COORD_WIDTH + 1 + (FRAC_W_WIDTH - FRAC_H_WIDTH)
else:
    print('H = W')
    GREATER_FRAC = FRAC_H_WIDTH
    MULT_H_0_EXT_CONCAT = 'MULT_H_0'
    MULT_H_1_EXT_CONCAT = ' + MULT_H_1;'
    SUM_H_WIDTH = MV_WIDTH + COORD_WIDTH + 1
    ###########################################################################
    MULT_V_0_EXT_CONCAT = 'MULT_V_0'
    MULT_V_1_EXT_CONCAT = ' + MULT_V_1;'
    SUM_V_WIDTH = MV_WIDTH + COORD_WIDTH + 1
    

#######################################################
# Generating an extended wire for MV inputs
#######################################################
MV_0_H_EXT = []
MV_0_V_EXT = []
for j in range(SUM_H_WIDTH-MV_WIDTH-GREATER_FRAC):
    MV_0_H_EXT.append('MV_0_H['+str(MV_WIDTH-1)+']')
    MV_0_V_EXT.append('MV_0_V['+str(MV_WIDTH-1)+']')
MV_0_H_EXT_CONCAT = ', '.join(MV_0_H_EXT)
MV_0_V_EXT_CONCAT = ', '.join(MV_0_V_EXT)


# print(MULT_H_0_EXT_CONCAT)
# print(MULT_H_1_EXT_CONCAT)
# print(MV_0_H_EXT_CONCAT)
# print(MV_0_V_EXT_CONCAT)
# print(REPRESENTATION_CONCAT)

Width of the fractional portion for W: 2 bits
Width of the fractional portion for H: 7 bits
H > W


In [85]:
###############################################
# Text writing: Mux generic
###############################################

mv_gen_6_text = '''/*--------------------------------------------------------------------
 * File: MV_gen_6_param.v
 * Date generated: 2023-04-12
 * Date modified: '''+str(date.today())+'''
 * Author: Bruna Suemi Nagai
 * Description: Motion Vectors generator for 4-parameters
 *------------------------------------------------------------------------------ */

module MV_gen_6_param (
	X_COORD,				// The X coordinate of the sample to be reconstructed [Fast Affine ME for VVC, Park S.]
	Y_COORD,				// The Y coordinate of the sample to be reconstructed [Fast Affine ME for VVC, Park S.]	
	MV_0_H,					// Horizontal component for MV0
	MV_1_H,					// Horizontal component for MV1	
    MV_2_H,					// Horizontal component for MV2	
	MV_0_V,					// Vertical component for MV0
	MV_1_V,					// Vertical component for MV1	
    MV_2_V,					// Vertical component for MV2	
	MV_H_OUT,				// Horizontal component for the resultant MV
	MV_V_OUT				// Vertical component for the resultant MV	
);


// ------------------------------------------
// IO declaration
// ------------------------------------------
	input signed ['''+str(COORD_WIDTH-1)+''':0] X_COORD;
	input signed ['''+str(COORD_WIDTH-1)+''':0] Y_COORD;
	
	input signed ['''+str(MV_WIDTH-1)+''':0] MV_0_H;
	input signed ['''+str(MV_WIDTH-1)+''':0] MV_1_H;
    input signed ['''+str(MV_WIDTH-1)+''':0] MV_2_H;
	
	input signed ['''+str(MV_WIDTH-1)+''':0] MV_0_V;
	input signed ['''+str(MV_WIDTH-1)+''':0] MV_1_V;
    input signed ['''+str(MV_WIDTH-1)+''':0] MV_2_V;

	output wire signed ['''+str(SUM_H_WIDTH)+''':0] MV_H_OUT;
	output wire signed ['''+str(SUM_H_WIDTH)+''':0] MV_V_OUT;
	

// ------------------------------------------
// Signals definitions
// ------------------------------------------
	wire signed ['''+str(MV_WIDTH-1)+''':0] MINUS_MV_0_H;
	wire signed ['''+str(MV_WIDTH-1)+''':0] MINUS_MV_0_V;

	wire signed ['''+str(MV_WIDTH)+''':0] SUB_H_0;
    wire signed ['''+str(MV_WIDTH)+''':0] SUB_H_1;

	wire signed ['''+str(MV_WIDTH)+''':0] SUB_V_0;
    wire signed ['''+str(MV_WIDTH)+''':0] SUB_V_1;
	
	wire signed ['''+str(MV_WIDTH+COORD_WIDTH)+''':0] MULT_H_0;		// 9 bits * 8 bits = 17 bits
	wire signed ['''+str(MV_WIDTH+COORD_WIDTH)+''':0] MULT_H_1;		// 9 bits * 8 bits = 17 bits
    
	wire signed ['''+str(MV_WIDTH+COORD_WIDTH)+''':0] MULT_V_0;		// 9 bits * 8 bits = 17 bits
	wire signed ['''+str(MV_WIDTH+COORD_WIDTH)+''':0] MULT_V_1;		// 9 bits * 8 bits = 17 bits
	
	wire signed ['''+str(SUM_H_WIDTH-1)+''':0] SUM_H;
	wire signed ['''+str(SUM_V_WIDTH-1)+''':0] SUM_V;
	
	wire signed ['''+str(SUM_H_WIDTH-1)+''':0] MV_0_H_EXT;
	wire signed ['''+str(SUM_H_WIDTH-1)+''':0] MV_0_V_EXT;
	
	
// ------------------------------------------
// Logic
// ------------------------------------------
	assign MINUS_MV_0_H = ~MV_0_H + '''+str(MV_WIDTH)+''''b1;
	assign MINUS_MV_0_V = ~MV_0_V + '''+str(MV_WIDTH)+''''b1; 

	assign SUB_H_0 = MV_1_H + MINUS_MV_0_H;	
    assign SUB_H_1 = MV_2_H + MINUS_MV_0_H;

	assign SUB_V_0 = MV_1_V + MINUS_MV_0_V;
    assign SUB_V_1 = MV_2_V + MINUS_MV_0_V;
	
	assign MULT_H_0 = SUB_H_0 * X_COORD;
	assign MULT_H_1 = SUB_H_1 * Y_COORD; 
	
	assign MULT_V_0 = SUB_V_0 * X_COORD;
	assign MULT_V_1 = SUB_V_1 * Y_COORD; 

	// Considerar que na divisão por '''+str(W)+''', o ponto está fixo em '''+REPRESENTATION_W_CONCAT+''' (>> '''+str(FRAC_W_WIDTH)+''')
	// Considerar que na divisão por '''+str(H)+''', o ponto está fixo em '''+REPRESENTATION_H_CONCAT+''' (>> '''+str(FRAC_H_WIDTH)+''')
    
	assign SUM_H = '''+ MULT_H_0_EXT_CONCAT + MULT_H_1_EXT_CONCAT + '''  
	assign MV_0_H_EXT = {'''+MV_0_H_EXT_CONCAT+''', MV_0_H, '''+str(GREATER_FRAC)+''''b0};
	
	assign SUM_V = '''+ MULT_V_0_EXT_CONCAT + MULT_V_1_EXT_CONCAT + ''' 
	assign MV_0_V_EXT = {'''+MV_0_V_EXT_CONCAT+''', MV_0_V, '''+str(GREATER_FRAC)+''''b0};
	
	assign MV_H_OUT = SUM_H + MV_0_H_EXT;
	assign MV_V_OUT = SUM_V + MV_0_V_EXT;
	
endmodule
'''

print(mv_gen_6_text)

/*--------------------------------------------------------------------
 * File: MV_gen_6_param.v
 * Date generated: 2023-04-12
 * Date modified: 2023-04-13
 * Author: Bruna Suemi Nagai
 * Description: Motion Vectors generator for 4-parameters
 *------------------------------------------------------------------------------ */

module MV_gen_6_param (
	X_COORD,				// The X coordinate of the sample to be reconstructed [Fast Affine ME for VVC, Park S.]
	Y_COORD,				// The Y coordinate of the sample to be reconstructed [Fast Affine ME for VVC, Park S.]	
	MV_0_H,					// Horizontal component for MV0
	MV_1_H,					// Horizontal component for MV1	
    MV_2_H,					// Horizontal component for MV2	
	MV_0_V,					// Vertical component for MV0
	MV_1_V,					// Vertical component for MV1	
    MV_2_V,					// Vertical component for MV2	
	MV_H_OUT,				// Horizontal component for the resultant MV
	MV_V_OUT				// Vertical component for the resultant MV	
);


// ------------------------------------------
// I